##목적 : 단어를 쓰면 가사가 출력되는 모델을 만들어보자

- 데이터 불러오기

In [1]:
import glob
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = '/content/drive/MyDrive/AIFFEL/E-06/data_set/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I. LIFE.', '', '']


- 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

I. LIFE.
I.
SUCCESS.


In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> i . life . <end>',
 '<start> i . <end>',
 '<start> success . <end>',
 '<start> published in a masque of poets <end>',
 '<start> at the request of h . h . , the author s <end>',
 '<start> fellow townswoman and friend . <end>',
 '<start> success is counted sweetest <end>',
 '<start> by those who ne er succeed . <end>',
 '<start> to comprehend a nectar <end>',
 '<start> requires sorest need . <end>']

- 평가 데이터셋 분리

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
  # 12000단어를 기억할 수 있는 tokenizer를 만들꺼임
  # 우린는 이미 문장은 정제했으니 filter가 필요 x
  # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      num_words=12000,
      filters=' ',
      oov_token='<unk>'
  )
  # corpus를 이용해 tokenizer 내부의 단어장을 완성
  tokenizer.fit_on_texts(corpus)
  # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
  tensor = tokenizer.texts_to_sequences(corpus)
  # 입력 데이터의 시퀀스 길이를 일정하게 맞춤
  # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
  # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
  # maxlen을 16으로 해서 과도한 padding 방지
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=16)

  print(tensor, tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    5   20 ...    0    0    0]
 [   2    5   20 ...    0    0    0]
 [   2 2762   20 ...    0    0    0]
 ...
 [ 130    5   22 ...   10 1019    3]
 [   5   37   15 ...  876  644    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fcbfabac050>


In [6]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:3, :10])

[[   2    5   20  102   20    3    0    0    0    0]
 [   2    5   20    3    0    0    0    0    0    0]
 [   2 2762   20    3    0    0    0    0    0    0]]


In [7]:
# 텐서 데이터는 모두 정수로 이루어짐
# 이 숫자는 tokenizer에 구축된 단어 사전 인덱스
# 단어 사전이 어떻게 구성되었는지 확인
for idx in tokenizer.index_word:
  print(idx, ':', tokenizer.index_word[idx])

  if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
from IPython.lib.display import ScribdDocument
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]
# tensor에서 <start>를 잘라내 타겟 문장 생성
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2   5  20 102  20   3   0   0   0   0   0   0   0   0   0]
[  5  20 102  20   3   0   0   0   0   0   0   0   0   0   0]


In [9]:
#  데이터셋 객체를 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터 셋을 만듦
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 15), dtype=tf.int32, name=None), TensorSpec(shape=(256, 15), dtype=tf.int32, name=None))>

In [10]:
# 훈련 데이터와 평가 데이터를 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=7)

In [11]:
# 훈련 데이터 확인
enc_train.shape

(140788, 15)

- 인공지능 학습시키기

In [12]:
# 모델 학습 준비
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 2048
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[-6.5123494e-04, -1.6956308e-04, -4.7167705e-04, ...,
          4.9701188e-04, -2.0004326e-04,  1.2820296e-04],
        [-6.9413771e-04, -7.4705982e-04, -3.3177590e-04, ...,
          1.0930588e-03, -4.2499229e-04,  2.9746635e-04],
        [-3.1701796e-04, -4.9526471e-04, -4.6928591e-04, ...,
          4.9289630e-04, -8.0005592e-04,  4.2378527e-04],
        ...,
        [-8.7332487e-04,  3.8362251e-04, -2.2249192e-03, ...,
          4.3192105e-03,  5.7042803e-04, -1.9135049e-03],
        [-2.9435314e-04,  2.9590441e-04, -2.6041986e-03, ...,
          4.3828301e-03,  8.5326860e-04, -1.9833366e-03],
        [-5.0612155e-04,  4.9903040e-04, -2.3827278e-03, ...,
          4.7529074e-03,  1.2391321e-03, -1.7811675e-03]],

       [[-6.5123494e-04, -1.6956308e-04, -4.7167705e-04, ...,
          4.9701188e-04, -2.0004326e-04,  1.2820296e-04],
        [-1.5486372e-03, -5.5365279e-05, -1.1753297e-03, ...,
          4.9253390e-04, 

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24578048  
                                                                 
 lstm (LSTM)                 multiple                  33562624  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  24590049  
                                                                 
Total params: 116,293,345
Trainable params: 116,293,345
Non-trainable params: 0
_________________________________________________________________


- 모델 학습

In [15]:
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=4)

Epoch 1/4
4400/4400 [==============================] - 333s 75ms/step - loss: 2.8799 - val_loss: 2.5925
Epoch 2/4
4400/4400 [==============================] - 327s 74ms/step - loss: 2.3236 - val_loss: 2.3523
Epoch 3/4
4400/4400 [==============================] - 327s 74ms/step - loss: 1.9010 - val_loss: 2.2387
Epoch 4/4
4400/4400 [==============================] - 327s 74ms/step - loss: 1.5734 - val_loss: 2.1981


In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you so much , i love you so much <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i hate")

'<start> i hate to see her leave sometimes <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i have")

'<start> i have been here with you <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> what is")

'<start> what is the answer , what are you waiting for ? <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> why so")

'<start> why so ray and aunt <unk> always <unk> <end> '

진행 과정
  - 데이터 불러오기
  - 데이터 정제
    - 조건 중 '토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기' 라는 조건이 있었음.
    - 저 조건을 만족하기 위해 for sentence in raw_corpus: 에서 if len(sentence) >= 15 continue 를 사용
    - 하지만 데이터 셋이 뭔가 이상해짐
    - 그래서 맨 처음 데이터를 불러오고 raw_corpus를 raw_corpus[:15]로 만듦
    - 데이터 갯수가 너무 줄어듬
  - 평가 데이터 분리
    - 위에서 실패한 조건을 여기서 시도
    - tensor 변환할 때 maxlen = 15를 줌
    - 데이터 형태가 (~~:14)가 나옴
    - maxlen = 16으로 변경
    - 데이터 형태 (~~:15)
  - 모델 학습 준비
    - embedding_size, hidden_size 를 128, 256, 512, 1024, 2048 로 변경해가면 진행
    - epoch는 7로 진행
    - val_loss = 2.27 까지 도달
    - 2.2 이하로 내려야됨
    - 2048, 2048에서 4/7에서 2.1 도달 
    - 하지만 5/7에서 오버피팅 
    - epoch = 4로 진행 
    - val_loss = 2.19로 성공
  - 결과
    - i love -> <start> i love you so much , i love you so much <end> 
    - i hate -> <start> i hate to see her leave sometimes <end> 
    - i have -> <start> i have been here with you <end> 
    - what is -> <start> what is the answer , what are you waiting for ? <end> 
    - why so -> <start> why so ray and aunt <unk> always <unk> <end> 
  - 다른 시도 
    - 위의 조건은 권고 사항이므로 루브릭 기준엔 없음
    - 그래서 maxlen을 20으로 두고 진행
    - 큰 어려움 없이 val_loss 2.2 이하로 내려감


### 회고
- 조건을 만족하면서 val_loss를 2.2 이하로 내리는게 힘들었음
- 임베딩 사이즈와 히든 사이즈를 변경해가며 학습시킨 결과 사이즈가 크면 오버피팅이 잘나고 작으면 val_loss가 잘 안내려감